# CHAPER 2. 검색문제

## 2.1 DNA 검색
- nucleotide: A, C, G, T
- codon: 3개의 nucleotide의 조합
- 유전자(dna)는 다수의 codon으로 구성됨

### 2.1.1 DNA 정렬
- nucleontide를 enum으로 정의한다
- enum 타입 중에서도 IntEnum을 사용한다.
  - IntEnum은 <u>비교연산자</u>를 사용할 수 있음

In [6]:
from enum import IntEnum
from typing import Tuple, List

# define type
Nucleotide: IntEnum = IntEnum('Nucleotide', ('A', 'C', 'G', 'T'))
Codon = Tuple[Nucleotide, Nucleotide, Nucleotide]
Gene = List[Codon]

gene_str: str = "ACGTGGCTCTCTAACGTACGTACGTACGGGGTTTATATATACCCTAGGACTCCCTTT"

def string_to_gene(s: str) -> Gene:
    gene: Gene = []
    for i in range(0, len(s), 3):
        # 현재 위치 다음에 2개의 문자가 없으면
        # Codon으로 변환할 수 없으므로 
        # for loop을 중지한다.
        if (i + 2) >= len(s):
            break
        
        codon: Codon = (Nucleotide[s[i]], Nucleotide[s[i + 1]], Nucleotide[s[i + 2]])
        gene.append(codon)
    return gene

In [ ]:
my_gene: Gene = string_to_gene(gene_str)
print(my_gene)

### 2.1.2 선형검색
- 찾고자 하는 요소가 발견되거나 자료구조의 끝에 도달할 때까지 순서대로 모든 요소를 검색
- 가장 간단하고, 자연스럽고, 명백한 방법
- 최악의 경우 자료구조의 모든 요소를 거쳐야 하므로 시간복잡도는 O(n)

In [7]:
def linear_contains_strict(gene: Gene, key_codon: Codon) -> bool:
    for codon in gene:
        if codon == key_codon:
            return True
    return False

In [ ]:
acg: Codon = (Nucleotide.A, Nucleotide.C, Nucleotide.G)
gat: Cogon = (Nucleotide.G, Nucleotide.A, Nucleotide.T)
print(linear_contains_strict(my_gene, acg))
print(linear_contains_strict(my_gene, gat))

>> 파이썬 내장 시퀀스 타입(list, tuple, range)는 모두 `__contains__()` 특수 메소드를 구현한다. 이 메소드를 사용하면 `in` 연산자를 사용하여 특정 항목을 검색할 수 있다.

### 2.1.3 이진 검색(binary search)
- 선형 검색보다 빠른 검색 방법, `O(logN)`
- 조건
    - 자료구조가 정렬되어 있어야 한다.
    - 인덱스로 접근 가능해야 한다.

In [14]:
# examples of binary search

def binary_search_words(words, target) -> int:
    start, end = 0, len(words) - 1
    while start <= end:
        mid = (start + end) // 2
        if words[mid] < target:
            start = mid + 1
        elif words[mid] > target:
            end = mid - 1
        else:
            return mid
    return -1

answer = binary_search_words(
    ["cat", "dog", "kangaroo", "llama", "rabbit", "rat", "zebra"],
    "rat"
)
print(answer)

5


In [ ]:
def binary_contains_strict(gene: Gene, key_codon: Codon) -> bool:
    low: int = 0
    high: int = len(gene) - 1
    while low <= high:
        mid: int = (low + high) // 2
        if gene[mid] < key_codon:
            low = mid + 1
        elif gene[mid] > key_codon:
            high = mid - 1
        else:
            return True
    return False

In [15]:
# python 표준 라이브러리 bisect 모듈을 사용하여 binary search 를 수행할 수 있다.
from bisect import bisect

def binary_contains_bisect(gene: Gene, key_codon: Codon) -> bool:
    return bisect(gene, key_codon)

### 2.1.4 제네릭 검색 예제
- `linear_contains()`와 `binary_contains()` 함수가 모든 type에 대해 동작하도록 일반화 하기
- python < 3.8 (아마도 3.10 인듯..)
  - `typing_extensions` 패키지를 설치해야한다.
  - `pip install typing_extensions`
- `generic_search`에 정의함

## 2.2 미로 찾기

In [2]:
from enum import Enum
from typing import List, NamedTuple, Callable, Optional
import random
from math import sqrt
from generic_search import dfs, node_to_path, astar, Node

class Cell(str, Enum):
    EMPTY   = " "
    BLOCKED = "X"
    START   = "S"
    GOAL    = "G"
    PATH    = "*"

# 미로의 개별 위치를 나타내기 위해
class MazeLocation(NamedTuple):
    row: int
    column: int

ImportError: cannot import name 'astar' from 'generic_search' (d:\personal\study\classic-cs\generic_search.py)

### 2.2.1 미로 무작위로 생성하기
- Maze class
  - 2차원 Cell List 생성
  - 행 수, 열 수, 시작위치, 목표 위치에 대한 인스턴스 변수
  - Blocked cell을 무작위로 생성한다.
- 무작위 생성 방법
  - 막힌 공간의 무작위 비율(임계값, sparseness: float = 0.2(default))을 설정한다.
  - 격자의 cell을 생성할 때 random 값을 생성하도록 하여 생성된 값이 sparseness보다 클 경우 BLCOKED Cell을 생성하도록 한다.
  - 이렇게 되면 통계적으로 막힌 공간의 분포는 임곗값의 비율과 비슷하게 된다.

### 2.2.2 기타 미로 세부사항
- 목표 지점에 도달했는지 확인하는 기능 `goal_test`
- 가능한 다음 위치 검색 `successors`
  - 상하좌우 위치를 확인하여 해당 위치에서 이동할 수 있는 빈 공간을 검색
  - 이동 가능한 모든 빈 공간(MazeLocation)의 리스트 반환

### 2.2.3 깊이 우선 탐색(Depth-First Search, DFS)
- 막다른 지점에 도달하여 최종 결정 지점으로 되돌아오기 전까지 가능한 깊이 탐색한다.
- Stack 이라는 자료구조에 의존한다.
  - LIFO(Last-In-First-Out)
  - 최소 `push`, `pop` 기능이 동작해야 한다.
- 구현 방법
  - 탐색 방문하려고 하는 장소 stack: `frontier`
  - 이미 방문할 장소 set: `explored`
  - `frontier` 변수에서 장소를 방문하면서 방문한 곳이 목표 지점인지 계속 확인
    - 목표 지점이라면 탐색을 종료한다.
  - `successors` 변수에서 현재 지점을 확인하여 다음 이동할 장소를 `frontier` 변수에 추가
  - `explored`에 방문할 장소가 있으면 방문하지 않는다.
  - `frontier` 변수가 비어있다면 모든 장소를 방문했다는 것이므로 탐색을 종료한다.
- `generic_search`에 정의함

In [21]:
class Maze:
    def __init__(self, rows: int = 10, columns: int = 10,
                 sparseness: float = 0.2,
                 start: MazeLocation = MazeLocation(0, 0),
                 goal: MazeLocation = MazeLocation(9, 9)) -> None:
        # 기본 인스턴스 변수 초기화
        self._rows: int = rows
        self._columns: int = columns
        self.start: MazeLocation = start
        self.goal: MazeLocation = goal
        # 격자를 빈 공간으로 채운다.
        self._randomly_fill(rows, columns, sparseness)
        # 시작 위치와 목표 위치를 설정한다.
        self._grid[start.row][start.column] = Cell.START
        self._grid[goal.row][goal.column] = Cell.GOAL

    def _randomly_fill(self, rows: int, columns: int, sparseness: float):
        for row in range(rows):
            for column in range(columns):
                if random.uniform(0, 1.0) < sparseness:
                    self.grid[row][column] = Cell.BLOCKED
    
    def __str__(self) -> str:
        output: str = ""
        for row in self._grid:
            output += "".join([c.value for c in row]) + "\n"
        return output

    def goal_test(self, ml: MazeLocation) -> bool:
        return ml == self.goal

    def successors(self, ml: MazeLocation) -> List[MazeLocation]:
        locations: List[MazeLocation] = []
        if ml.row + 1 < self._rows and self._grid[ml.row + 1][ml.column] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row + 1, ml.column))
        if ml.row - 1 >= 0 and self._grid[ml.row - 1][ml.column] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row - 1, ml.column))
        if ml.column + 1 < self._columns and self._grid[ml.row][ml.column + 1] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row, ml.column + 1))
        if ml.column - 1 >= 0 and self._grid[ml.row][ml.column - 1] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row, ml.column - 1))
        return locations
    
    def mark(self, path: List[MazeLocation]):
        for maze_location in path:
            self._grid[maze_location.row][maze_location.column] = Cell.PATH
        self._grid[self.start.row][self.start.column] = Cell.START
        self._grid[self.goal.row][self.goal.column] = Cell.GOAL

    def clear(self, path: List[MazeLocation]):
        for maze_location in path:
            self._grid[maze_location.row][maze_location.column] = Cell.EMPTY
        self._grid[self.start.row][self.start.column] = Cell.START
        self._grid[self.goal.row][self.goal.column] = Cell.GOAL

NameError: name 'MazeLocation' is not defined

In [ ]:
maze: Maze = Maze()
print(maze)

In [44]:
m: Maze = Maze()
print(m)
solution1: Optional[Node[MazeLocation]] = dfs(m.start, m.goal_test, m.successors)
if solution1 is None:
    print("깊이 우선 탐색으로 길을 찾을 수 없습니다.")
else:
    path1: List[MazeLocation] = node_to_path(solution1)
    m.mark(path1)
    print(m)
    m.clear(path1)

AttributeError: module 'generic_search' has no attribute 'node_to_path'

### 2.2.4 너비 우선 탐색(Breadth-First Search, BFS)
- 깊이 우선 탐색으로 찾은 목표 지점에 대한 경로는 부자연스럽게 보일 수 있으며 최단 경로가 아닐 수 있다.
- BFS는 탐색의 각 반복마다 출발 지점에서 한 계층의 노드를 가까운 지점부터 순차적으로 탐색함으로써 <u>항상 최단 경로를 찾는다.</u>
- 일반적으로 더 빨리 목표 지점을 찾음
- 최단경로(BFS), 빠른 탐색(DFS)
- 구현
  - Queue 필요, FIFO
  - python의 deque 이용할 수 있다.

### 2.2.5 A* 알고리즘
- BFS와 마찬가지로 출발 지점에서 목표 지점까지의 **최단 경로**를 찾는 것이 목표
- 비용 함수와 휴리스틱(heuristic) 함수를 사용

탐색을 고려하는 모든 지점에 대한 총 비용 f(n)은
f(n) = g(n) + h(n)
- 비용함수 g(n)
  - 특정 지점에 도달하기 위한 비용 확인
  - 미로찾기의 경우 한 지점으로 가기 위해 거처야 하는 점의 수
- 휴리스틱 함수 h(n)
  - 해당 지점에서 목표 지점까지 비용 추정
  - h(n)이 허용 가능한 휴리스틱(admissible heuristic) 이라면 발견된 경로를 최적으로 판단
    - 목표 지점에 도달하는데 드는 비용을 과대평가하지 않음
    - e.g. 2차원 평면에서 직선거리 휴리스틱
A* 알고리즘은 방문하지 않은 지점에서 다음 지점을 선택할 때 f(n)이 가장 낮은 것을 선택함
- f(n)의 값을 우선순위 큐에 넣어 가장 낮은 f(n) 지점을 선택

우선순위 큐
- 이진 힙을 내부적으로 사용
- push, pop time complexity는 O(lg n)
- 파이썬에서는 heapq 사용
  - heapush와 heappop의 내부에서 사용하는 비교연산을 위해 Node class에 `__lt__()` 특수 메서드 구현

In [2]:
class PriorityQueue(Generic[T]):
    def __init__(self) -> None:
        self._container: List[T] = []
    
    @property
    def empty(self) -> bool:
        return not self._container
    
    def push(self, item: T) -> None:
        heappush(self._container, item)

    def pop(self) -> T:
        return heappop(self._container)

    def __repr__(self) -> str:
        return repr(self_container)

NameError: name 'Generic' is not defined